In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.pandas
import pickle

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
hv.extension("bokeh")

# Load data

In [ ]:
filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329flippedpwm2.pickle"
data = pickle.load(open(filename, "rb"))

In [ ]:
filename = "/n/groups/paulsson/jqs1/sigma-circuits/210329flippedpwm4.pickle"
data = pickle.load(open(filename, "rb"))

# Analysis

In [ ]:
data["area"].groupby(["pos", "t"]).mean().hvplot.line(by="pos")

In [ ]:
data["area"].groupby(["pos", "t"]).quantile(0.75).reset_index().hvplot.line(
    "t", "area", by="pos"
)

In [ ]:
mean_fluorescence = (
    data[data["area"].between(100, 450)]["mean_intensity"].groupby(["pos", "t"]).mean()
)

In [ ]:
mean_fluorescence.reset_index().hvplot.line("t", "mean_intensity", by="pos", logy=True)

In [ ]:
from astropy.stats import mad_std, sigma_clip

m_int = data[
    data["area"].between(100, 450) & data["mean_intensity"].between(100, 5000)
]["mean_intensity"].loc[[16, 53, 63, 95, 123, 149]]
dslice = m_int.groupby(["pos", "t"]).apply(
    lambda y: sigma_clip(y, sigma=2, maxiters=10, masked=False)
)
mf = dslice.apply(np.mean)

sf = dslice.apply(np.std)

r = pd.DataFrame({"stdm": mf - sf, "stdp": mf + sf})
means = mf.reset_index().hvplot.line("t", "mean_intensity", by="pos", logy=True)
errors = r.hvplot.area(
    x="t", y="stdm", y2="stdp", by="pos", stacked=False, alpha=0.2, logy=True
).opts(width=1000, height=500)
IPTG_starts = (
    hv.VLine(12).opts(alpha=0.5, color="blue")
    * hv.VLine(60).opts(alpha=0.5, color="blue")
    * hv.VLine(108).opts(alpha=0.5, color="blue")
    * hv.VLine(156).opts(alpha=0.5, color="blue")
)
EZ_starts = (
    hv.VLine(30).opts(alpha=0.5, color="grey")
    * hv.VLine(78).opts(alpha=0.5, color="grey")
    * hv.VLine(126).opts(alpha=0.5, color="grey")
    * hv.VLine(174).opts(alpha=0.5, color="grey")
)
aTc_starts = (
    hv.VLine(42).opts(alpha=0.5, color="red")
    * hv.VLine(90).opts(alpha=0.5, color="red")
    * hv.VLine(138).opts(alpha=0.5, color="red")
    * hv.VLine(186).opts(alpha=0.5, color="red")
)

means * errors * IPTG_starts * EZ_starts * aTc_starts

In [ ]:
rlabels = ["A", "B", "C", "D", "E", "F", "G", "H"]


def rgroup(pos):
    return rlabels[int(np.floor(pos / 20))]


df = data.reset_index()
df["pos"] = df["pos"].apply(rgroup)

In [ ]:
from astropy.stats import mad_std, sigma_clip

m_int = df[
    df["area"].between(300, 1500)
    & df["GFP-PENTA"].between(100, 5000)
    & df["pos"].isin(["A", "C", "D", "E", "G", "H"])
][["GFP-PENTA", "pos", "t"]]
dslice = m_int.groupby(["pos", "t"]).apply(
    lambda y: sigma_clip(y, sigma=2, maxiters=10, masked=False)
)
mf = dslice.apply(np.mean)
sf = dslice.apply(np.std)

r = pd.DataFrame({"stdm": mf - sf, "stdp": mf + sf})
means = mf.hvplot.line("t", by="pos", logy=True)
errors = r.hvplot.area(
    x="t", y="stdm", y2="stdp", by="pos", stacked=False, alpha=0.2, logy=True
)

line_alpha = 0.2
IPTG_starts = (
    hv.VLine(12).opts(alpha=line_alpha, color="blue")
    * hv.VLine(60).opts(alpha=line_alpha, color="blue")
    * hv.VLine(108).opts(alpha=line_alpha, color="blue")
    * hv.VLine(156).opts(alpha=line_alpha, color="blue")
)
EZ_starts = (
    hv.VLine(30).opts(alpha=line_alpha, color="grey")
    * hv.VLine(78).opts(alpha=line_alpha, color="grey")
    * hv.VLine(126).opts(alpha=line_alpha, color="grey")
    * hv.VLine(174).opts(alpha=line_alpha, color="grey")
)
aTc_starts = (
    hv.VLine(42).opts(alpha=line_alpha, color="red")
    * hv.VLine(90).opts(alpha=line_alpha, color="red")
    * hv.VLine(138).opts(alpha=line_alpha, color="red")
    * hv.VLine(186).opts(alpha=line_alpha, color="red")
)

# (IPTG_starts*EZ_starts*aTc_starts*means*errors).opts(width=1000,height=400)
(means * errors).opts(width=1000, height=400)